#Applied Natural Language Processing 955G5

##Computer Based Examination, 2026

Remember, you can add cells and change their type (between code and text/markdown) as required to answer the questions.

In [ ]:
# update your candidate number here
candidate_number = 291065

#Question 1 (50 Marks)

This question is about document similarity and information retrieval.

In [80]:
### do not change the code in this cell
# make sure you run this cell

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from math import log2, sqrt

nltk.download('stopwords')
nltk.download("punkt")
nltk.download('stopwords')
nltk.download("punkt_tab")
stop = stopwords.words('english')



# This is corpus of lyrics, artists and song titles which we will use in this question.
corpus = [("Get up, stand up. Stand up for your right. Get up, stand up. Don't give up the fight.", "Bob Marley", "Get Up, Stand Up"),
          ("Cause all around the gold is glistening. Making sure it keeps us hypnotized.", "John Martyn", "Don't Want to Know"),
          ("You're told that you must. And you must earn a living. And you must earn a crust. And be like everybody else.", "Reverend and the Makers", "Heavyweight Champion of the World"),
          ("Another year and then you'd be happy. Just one more year and then you'd be happy. But you're cryin', you're cryin' now.", "Gerry Rafferty", "Baker Street"),
          ("You only live twice. Or so it seems. One life for yourself. And one for your dreams.", "Nancy Sinatra", "You Only Live Twice"),
          ("For what is a man, what has he got? If not himself, then he has naught.", "Frank Sinatra", "My Way")]

# This is a query that we will retrieve relevant documents for.
query = "In life, what have we got to stand up for?"

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


a) Preprocess the information in the list `corpus` by following the steps below.

i) Create three dictionaries - `lyrics`, `artists` and `titles` - corresponding to the three elements in each tuple of `corpus`. Each key in these dictionaries should be the position in the original list and each value should be a string from the tuple at that position in the list.

Each value in `lyrics` should be item 0 from a tuple, i.e. the lyric itself. Each value in `artists` should be item 1 from the tuple, i.e. the artist associated with the lyric. Each value in `titles` should be item 2 in a tuple, i.e. the song title of the lyric.

So for example, the example corpus `[("Freedom", "Aretha", "Think")]` would be broken into three dictionaries: `{0: "Freedom"}`, `{0: "Aretha"}` and `{0: "Think"}`.

(7 marks)

In [17]:
lyrics, artists, titles = {}, {}, {}

for index,sent in enumerate(corpus):
    lyrics[index] = sent[0]
    artists[index] = sent[1]
    titles[index] = sent[2]

print(lyrics[0])
print(artists[0])
print(titles[0])

Get up, stand up. Stand up for your right. Get up, stand up. Don't give up the fight.
Bob Marley
Get Up, Stand Up


ii) Tokenise the quotation strings in the dictionary `lyrics` to produce a new dictionary,  called `tokenised`, with the same keys, in which each value is a list of tokens.

So, for example the example dictionary `{0: "Choose life."}` would become `{0: ["Choose", "life", "."]}`.

(4 marks)

In [19]:
tokenised = {}
for key in lyrics.keys():
    current = lyrics.get(key)
    current = word_tokenize(current)
    tokenised[key] = current
tokenised[0]

['Get',
 'up',
 ',',
 'stand',
 'up',
 '.',
 'Stand',
 'up',
 'for',
 'your',
 'right',
 '.',
 'Get',
 'up',
 ',',
 'stand',
 'up',
 '.',
 'Do',
 "n't",
 'give',
 'up',
 'the',
 'fight',
 '.']

iii) Case normalise the tokenised strings and remove stopwords and punctuation, putting the results into a new dictionary called `normalised`, with the same keys as `tokenised` and the values being normalised lists of tokens.

(6 marks)

In [27]:
normalised = {}

for key in tokenised.keys():
    current = tokenised.get(key)
    normed = [w.lower() for w in current if w.isalpha() and w not in stop]
    normalised[key] = normed
normalised[0] 

['get', 'stand', 'stand', 'right', 'get', 'stand', 'do', 'give', 'fight']

iv) Describe two other forms of pre-processing that could be applied to text documents.

(4 marks)

Another form of normalisation could be number normalisation. This is where we convert any reference to a number to the string 'NUM'. We do this because numbers are sparse features. For example, even if we were referring to semi-modern history there are still 100 of numbers to represent the years for the past few centuries. In terms of being a token, a year may only come up once across several documents. Instead, just understanding the context that this token is a number can be enough and allows all numbers to be groups removing the issue of sparseness. An example would be turning "1980" to "NUM". 

Another form is Lemmatisation. This is where we reduce works to their dictionary form, i.e. their lemma, which is the derivative of their past-of-speech. It is similar to another form of normalisation called stemming which reduces a word by chopping off its pre-fix or suffix but is slightly more advanced because the remaining root word will always be a true dictionary word. This is important for many NLP models and applications where results and methods need to be interpretable. This type of normalisation is also very helpful for feature sparsity as it allows mutual words that are derived from the same meaning to be grouped under the same feature, thus reducing the feature sparseness.

b) Convert each document, ie each list of tokens, into a tfidf representation by following the steps below.

i) Calculate document frequencies for each token found in the documents contained in `normalised` and put the results in a dictionary called `df`. That is count how many different entries in the dictionary each token is found in. For example, the token `"one"` occurs in two different sets of lyrics, so `df["one"]` should be 2.

(4 marks)

In [36]:
unique = set([token for key in normalised.keys() for token in normalised[key]])

df = {}

for token in unique:
    counter = 0
    for key in normalised.keys():
        if token in normalised[key]:
            counter += 1
    df[token] = counter
df["one"]

2

ii) Calculate inverse document frequencies from the document frequencies derived in the previous question and put the results in a dictionary called `idf`.

$$IDF(w) = \log_2 \left( \frac{N}{DF(w)} \right) $$

where $N$ is the total number of documents and $DF(w)$ is the document frequency of the word $w$.

(4 marks)

In [41]:
idf = {}

num_docs = len(normalised.keys())

for key in df.keys():
    current_df = df[key]
    _idf = log2(num_docs/current_df)
    idf[key]=_idf

idf

{'earn': 2.584962500721156,
 'keeps': 2.584962500721156,
 'you': 1.584962500721156,
 'get': 2.584962500721156,
 'fight': 2.584962500721156,
 'around': 2.584962500721156,
 'like': 2.584962500721156,
 'dreams': 2.584962500721156,
 'life': 2.584962500721156,
 'glistening': 2.584962500721156,
 'but': 2.584962500721156,
 'or': 2.584962500721156,
 'happy': 2.584962500721156,
 'year': 2.584962500721156,
 'man': 2.584962500721156,
 'stand': 2.584962500721156,
 'gold': 2.584962500721156,
 'must': 2.584962500721156,
 'hypnotized': 2.584962500721156,
 'just': 2.584962500721156,
 'if': 2.584962500721156,
 'seems': 2.584962500721156,
 'one': 1.584962500721156,
 'live': 2.584962500721156,
 'us': 2.584962500721156,
 'another': 2.584962500721156,
 'living': 2.584962500721156,
 'cryin': 2.584962500721156,
 'twice': 2.584962500721156,
 'everybody': 2.584962500721156,
 'for': 2.584962500721156,
 'crust': 2.584962500721156,
 'right': 2.584962500721156,
 'sure': 2.584962500721156,
 'told': 2.58496250072115

iv) Convert each document in `normalised` from a list of tokens to a dictionary of term frequencies and put the results in a dictionary called `tf`.

The keys of `tf` should be positions in the original list `corpus` and the values should be the term frequency dictionaries, which map from tokens to frequency in each document.

So, for example, `{0: ["Hello", "Bob"]}` would become `{0: {"Hello": 1, "Bob": 1}}`.

(4 marks)

In [61]:
from nltk import FreqDist

tf = {}

for key in normalised.keys():
    tf[key] = dict(FreqDist(normalised[key]))

tf[0]

{'get': 2, 'stand': 3, 'right': 1, 'do': 1, 'give': 1, 'fight': 1}

v) Convert the raw term frequencies in `tf` to tfidf values using the dictionary `idf`.

(3 marks)

In [93]:
tfidf = {}

for key in normalised.keys():
    current = tf.get(key,{})
    current_tfidf = {} # avoid updating via direct ref
    for entry in current.keys():
        tf_value = current.get(entry)
        idf_value = idf.get(entry)
        tfidf_value = tf_value * idf_value
        current_tfidf[entry] = tfidf_value
    tfidf[key] = current_tfidf
tfidf

{0: {'get': 5.169925001442312,
  'stand': 7.754887502163468,
  'right': 2.584962500721156,
  'do': 2.584962500721156,
  'give': 2.584962500721156,
  'fight': 2.584962500721156},
 1: {'cause': 2.584962500721156,
  'around': 2.584962500721156,
  'gold': 2.584962500721156,
  'glistening': 2.584962500721156,
  'making': 2.584962500721156,
  'sure': 2.584962500721156,
  'keeps': 2.584962500721156,
  'us': 2.584962500721156,
  'hypnotized': 2.584962500721156},
 2: {'you': 1.584962500721156,
  'told': 2.584962500721156,
  'must': 7.754887502163468,
  'and': 4.754887502163468,
  'earn': 5.169925001442312,
  'living': 2.584962500721156,
  'crust': 2.584962500721156,
  'like': 2.584962500721156,
  'everybody': 2.584962500721156,
  'else': 2.584962500721156},
 3: {'another': 2.584962500721156,
  'year': 5.169925001442312,
  'happy': 5.169925001442312,
  'just': 2.584962500721156,
  'one': 1.584962500721156,
  'but': 2.584962500721156,
  'cryin': 5.169925001442312},
 4: {'you': 1.584962500721156,


c) In the following steps, preprocess the string `query` and convert it to a tfidf representation. Then use this to find relevant quotations in the index `tfidf`.

i) Define a function `dot` which takes two dictionaries containing tfidf values as inputs and calculates their dot product.

(3 marks)

In [64]:
def dot(dict1, dict2):
    _c = 0
    for word in dict1.keys():
        if word in dict2.keys():
            _c += dict1[word] * dict2[word]
    return _c

ii) Define a function, `sim`, which takes two dictionaries containing tfidf values as inputs and , using your `dot` function, calculates their cosine similarity.

(3 marks)

In [75]:
def sim(dict1, dict2):
    sim=dot(dict1,dict2)/(sqrt(dot(dict1,dict1)*dot(dict2,dict2)))
    return sim

print(sim(tfidf[0],tfidf[0]))
print(sim(tfidf[0],tfidf[1]))

1.0
0.0


iii) Preprocess the string `query` and convert it to a tfidf representation. Then calculate its cosine similarity to all the documents in the dictionary `tfidf`.

For any document with a non-zero similarity, print out the similarity and the corresponding lyric and title from the relevant dictionaries.

(8 marks)

In [ ]:
print(f"Query: {query}")
print("---")

# PREPROCCESS
q = word_tokenize(query)
q_norm = [t.lower() for t in q if t.isalpha() and t.lower() not in stop]
q_freq = dict(FreqDist(q_norm))

q_tfidf = {}

for key in q_freq.keys():
    _tf_q = q_freq.get(key)
    _idf_q = idf.get(key,0)
    _tfidf_q = _tf_q * _idf_q
    q_tfidf[key] = _tfidf_q

for song in tfidf.keys():
    score = sim(q_tfidf, tfidf[song])
    if score > 0:
        print(f"Title: {titles[song]} | Artist: {artists[song]}")
        print(f"Has a Cosine Similarity of {score:.4f}")
        print(f'"{lyrics[song]}"')
        print("---")

Query: In life, what have we got to stand up for?
---
Title: Get Up, Stand Up | Artist: Bob Marley
Has a Cosine Similarity of 0.4201
"Get up, stand up. Stand up for your right. Get up, stand up. Don't give up the fight."
---
Title: You Only Live Twice | Artist: Nancy Sinatra
Has a Cosine Similarity of 0.2009
"You only live twice. Or so it seems. One life for yourself. And one for your dreams."
---
Title: My Way | Artist: Frank Sinatra
Has a Cosine Similarity of 0.2582
"For what is a man, what has he got? If not himself, then he has naught."
---
